<a href="https://colab.research.google.com/github/hosseiniii/Comments-with-Sentiment-Label/blob/main/Text_Preprocess_and_Vectorzie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/dirve')

Mounted at /content/dirve


In [2]:
cd /content/dirve/MyDrive/BFP

/content/dirve/MyDrive/BFP


## Install required packages

In [34]:
!pip install -q hazm
!pip install -q clean-text[gpl]
!pip install -q fasttext

     |████████████████████████████████| 68 kB 4.3 MB/s 


## Loading the required libraries

In [35]:
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import fasttext

# Text Cleaning
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from cleantext import clean
import hazm

# Visualization
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Loading Datasets

In [5]:
unlabeled_comments_df = pd.read_excel("comment.xlsx")
unlabeled_comments_df.drop(["product_id", "confirmed_at"], axis=1, inplace=True)
unlabeled_comments_df = unlabeled_comments_df.rename(columns={'comment': 'text'})
unlabeled_comments_df.head()

,text
0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
1,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...
2,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...
3,افا از وقتی من اینو واسه پسرم خریدم همه بچه ها...
4,این تیغ محصول کشور آمریکا هست ( در عکس محصول م...


In [6]:
digi1_dataset = pd.read_csv("dataset-digi-1ta3.csv")
digi2_dataset = pd.read_csv("digi2.csv")
sentipers_dataset = pd.read_csv("sentipers.csv")


digi1_dataset.drop("Score", axis=1, inplace=True)
digi2_dataset.drop("Unnamed: 0", axis=1, inplace=True)
sentipers_dataset.drop(["Unnamed: 0", "index", "sid", "file"], axis=1, inplace=True)

digi1_dataset = digi1_dataset.rename(columns={'Text': 'text', 'Suggestion': 'value'})
sentipers_dataset = sentipers_dataset.rename(columns={'polarity': 'value'})


labeled_comments_df = pd.DataFrame(data=digi1_dataset)
labeled_comments_df = labeled_comments_df.append(digi2_dataset, ignore_index=True)
labeled_comments_df = labeled_comments_df.append(sentipers_dataset, ignore_index=True)

labeled_comments_df.head()

,text,value
0,این اولین تجربه من برای خرید ایفون هست امروز...,1
1,خرید این محصول رو توصیه میکنم,1
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,1
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,1
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,1


In [7]:
print("1. Positive \n2. neutral \n3. Negative\n")
labeled_comments_df['value'].value_counts()

1. Positive 
2. neutral 
3. Negative



1    10841
2     6607
3     2536
Name: value, dtype: int64

## Text Preprocessing

In [8]:
labeled_comments_df['comment_len_by_words'] = labeled_comments_df['text'].apply(lambda t: len(hazm.word_tokenize(t)))
labeled_comments_df.head()

,text,value,comment_len_by_words
0,این اولین تجربه من برای خرید ایفون هست امروز...,1,24
1,خرید این محصول رو توصیه میکنم,1,6
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,1,11
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,1,29
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,1,55


In [10]:
unlabeled_comments_df['comment_len_by_words'] = unlabeled_comments_df['text'].apply(lambda t: len(hazm.word_tokenize(str(t))))
unlabeled_comments_df.head()

,text,comment_len_by_words
0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,221
1,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...,16
2,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...,43
3,افا از وقتی من اینو واسه پسرم خریدم همه بچه ها...,81
4,این تیغ محصول کشور آمریکا هست ( در عکس محصول م...,87


In [13]:
min_max_len = labeled_comments_df["comment_len_by_words"].min(), labeled_comments_df["comment_len_by_words"].max()
print(f'Labeled Min Words: {min_max_len[0]} \tLabeled Max Words: {min_max_len[1]}')

min_max_len = unlabeled_comments_df["comment_len_by_words"].min(), unlabeled_comments_df["comment_len_by_words"].max()
print(f'Unlabeled Min Words: {min_max_len[0]} \tUnlabeled Max Words: {min_max_len[1]}')

Labeled Min Words: 1 	Labeled Max Words: 1243
Unlabeled Min Words: 1 	Unlabeled Max Words: 2170


In [14]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='comment_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [15]:
data_gl_than(labeled_comments_df, 256, 3)
data_gl_than(unlabeled_comments_df, 256, 3)

Texts with word length of greater than 3 and less than 256 includes 96.99% of the whole!
Texts with word length of greater than 3 and less than 256 includes 95.33% of the whole!


In [16]:
minlim, maxlim = 3, 256

In [17]:
# remove comments with the length of fewer than three words
labeled_comments_df['comment_len_by_words'] = labeled_comments_df['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
labeled_comments_df = labeled_comments_df.dropna(subset=['comment_len_by_words'])
labeled_comments_df = labeled_comments_df.reset_index(drop=True)

unlabeled_comments_df['comment_len_by_words'] = unlabeled_comments_df['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
unlabeled_comments_df = unlabeled_comments_df.dropna(subset=['comment_len_by_words'])
unlabeled_comments_df = unlabeled_comments_df.reset_index(drop=True)

In [20]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=labeled_comments_df['comment_len_by_words'],
    name="Labeled Comments" 
))

fig.add_trace(go.Histogram(
    x=unlabeled_comments_df['comment_len_by_words'],
    name="Unlabeled Comments" 
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [21]:
fig = go.Figure()

groupby_rate = labeled_comments_df.groupby('value')['value'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_rate.index)),
    y=groupby_rate.tolist(),
    text=groupby_rate.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of Comment Scores within comments',
    xaxis_title_text='Score',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [23]:
def rate_to_label(rate):
    if rate == 1:
        return 'positive'
    elif rate == 2:
      return 'neutral'
    else:
        return 'negative'


labeled_comments_df['label'] = labeled_comments_df['value'].apply(lambda t: rate_to_label(t))
labels = list(sorted(labeled_comments_df['label'].unique()))
labeled_comments_df.head()


,text,value,comment_len_by_words,label
0,این اولین تجربه من برای خرید ایفون هست امروز...,1,24.0,positive
1,خرید این محصول رو توصیه میکنم,1,6.0,positive
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,1,11.0,positive
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,1,29.0,positive
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,1,55.0,positive




*   fixing unicodes
*   removing specials like a phone number, email, url, new lines, ...
*   cleaning HTMLs
*   normalizing
*   removing emojis

In [26]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [28]:
# cleaning labeled comments
labeled_comments_df['cleaned_comment'] = labeled_comments_df['text'].apply(cleaning)


# calculate the length of comments based on their words
labeled_comments_df['cleaned_comment_len_by_words'] = labeled_comments_df['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))

# remove comments with the length of fewer than three words
labeled_comments_df['cleaned_comment_len_by_words'] = labeled_comments_df['cleaned_comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else len_t)
data = labeled_comments_df.dropna(subset=['cleaned_comment_len_by_words'])
data = data.reset_index(drop=True)

data.head()

,text,value,comment_len_by_words,label,cleaned_comment,cleaned_comment_len_by_words
0,این اولین تجربه من برای خرید ایفون هست امروز...,1,24.0,positive,این اولین تجربه من برای خرید ایفون هست امروز ب...,24
1,خرید این محصول رو توصیه میکنم,1,6.0,positive,خرید این محصول رو توصیه میکنم,6
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,1,11.0,positive,۱ ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ندیدم,11
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,1,29.0,positive,سلام خدمت دوستان این گوشی از همه نظر عالی کیفی...,29
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,1,55.0,positive,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,55


In [29]:
labeled_comments_df = labeled_comments_df[['cleaned_comment', 'label']]
labeled_comments_df.columns = ['comment', 'label']
labeled_comments_df.head()

,comment,label
0,این اولین تجربه من برای خرید ایفون هست امروز ب...,positive
1,خرید این محصول رو توصیه میکنم,positive
2,۱ ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ندیدم,positive
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیفی...,positive
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,positive


In [30]:
# cleaning unlabeled comments
unlabeled_comments_df['cleaned_comment'] = unlabeled_comments_df['text'].apply(cleaning)


# calculate the length of comments based on their words
unlabeled_comments_df['cleaned_comment_len_by_words'] = unlabeled_comments_df['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))

# remove comments with the length of fewer than three words
unlabeled_comments_df['cleaned_comment_len_by_words'] = unlabeled_comments_df['cleaned_comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else len_t)
data = unlabeled_comments_df.dropna(subset=['cleaned_comment_len_by_words'])
data = data.reset_index(drop=True)

data.head()

,text,comment_len_by_words,cleaned_comment,cleaned_comment_len_by_words
0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,221.0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,220
1,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...,16.0,بوی تند ولی خوشبو داره. ماندگاریش خوبه و هر چق...,16
2,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...,43.0,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...,42
3,افا از وقتی من اینو واسه پسرم خریدم همه بچه ها...,81.0,افا از وقتی من اینو واسه پسرم خریدم همه بچه‌ها...,76
4,این تیغ محصول کشور آمریکا هست ( در عکس محصول م...,87.0,این تیغ محصول کشور آمریکا هست (در عکس محصول مش...,87


In [33]:
unlabeled_comments_df = unlabeled_comments_df[['cleaned_comment']]
unlabeled_comments_df.columns = ['comment']
unlabeled_comments_df.head()

,comment
0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
1,بوی تند ولی خوشبو داره. ماندگاریش خوبه و هر چق...
2,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...
3,افا از وقتی من اینو واسه پسرم خریدم همه بچه‌ها...
4,این تیغ محصول کشور آمریکا هست (در عکس محصول مش...


## Load PreTrain Fasttext model

In [36]:
%time
model = fasttext.load_model("cc.fa.300.bin")
print(len(model.get_sentence_vector('بوی تند ولی خوشبو داره. ماندگاریش خوبه و هر چقدر میگذره خوشبوتر میشه..')))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs
300


In [38]:
def get_comment_vector(comment):
  return model.get_sentence_vector(comment)

In [41]:
labeled_comments_df['vector'] = labeled_comments_df['comment'].apply(lambda t: get_comment_vector(t))
labeled_comments_df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,comment,label,vector
0,این اولین تجربه من برای خرید ایفون هست امروز ب...,positive,"[0.022520525, 0.05368863, -0.020799093, -0.013..."
1,خرید این محصول رو توصیه میکنم,positive,"[0.04413482, 0.05712284, -0.042128816, -0.0379..."


In [42]:
unlabeled_comments_df['vector'] = unlabeled_comments_df['comment'].apply(lambda t: get_comment_vector(t))
unlabeled_comments_df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,comment,vector
0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[0.020869812, 0.053931676, -0.012462843, 0.000..."
1,بوی تند ولی خوشبو داره. ماندگاریش خوبه و هر چق...,"[-0.0105422335, 0.060699597, -0.014465652, -0...."


## Save Dataframes to csv file

In [ ]:
labeled_comments_df.to_csv("labeled_comments.csv")
unlabeled_comments_df.to_csv("unlabeled_comments.csv")